<a href="https://colab.research.google.com/github/felipemaiapolo/analise_sentimentos_PMR3508/blob/main/analise_de_sentimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de sentimentos utilizando representação Doc2Vec para textos

#### Disciplina: PMR3508 - Aprendizado de Máquina e Reconhecimento de Padrões

Neste notebook será apresentado como fazer uso de um modelo Doc2Vec pré-treinado para a análise de sentimentos. O modelo Doc2Vec foi introduzido em 2014 por [0] e tem duas possíveis formulações, sendo que aqui focaremos naquela análoga ao Word2Vec COBW, que se chama PV-DM (Distributed Memory version of Paragraph Vector).

O modelo Doc2Vec é um ótimo exemplo de como as Redes Neurais Artificiais podem ser utilizadas para gerar representações estruturadas para dados, a princípio, não estruturados. Como já foi dito, a versão mais popular do modelo Doc2Vec é uma extensão do famigerado modelo Word2Vec, que é amplamente utilizado para gerar representações para palavras e expressões. É importante dizer que os treinamentos tanto do Word2Vec quanto do Doc2Vec são dados de maneira auto supervisionada (*self-supervised*), que resumidamente é se fazer o uso de métodos supervisionados tradicionais para a realização de tarefas tradicionalmente consideradas de aprendizado não supervisionado. O aprendizado auto supervisionado é um paradigma moderno de aprendizagem e vale a pena pesquisar mais sobre.

Escolhemos utilizar as representações Doc2Vec para textos neste trabalho pois, apesar de não ser o estado da arte no que tange à utilização de redes neurais para processamento de linguagem natural, temos os seguintes benefícios didáticos e práticos: (i) esse modelo é uma extensão de um dos modelos de redes neurais históricamente mais importantes para NLP, que é o Word2Vec; (ii) é um modelo relativamente simples e que dá uma boa ideia de como as Redes Neurais podem ser utilizadas para se trabalhar com dados não estruturados, e.g. textos; (iii) é um modelo de fácil/rápido treinamento e uso, o que possibilita a obtenção de um bom baseline para seus projetos.

Agora faremos uma breve introdução de como utilizar um modelo Doc2Vec pré-treinado em seus projetos de Machine Learning e NLP.

### Base de dados que utilizaremos

Os dados utilizados neste notebook podem ser encontradas em https://ai.stanford.edu/~amaas/data/sentiment/ em sua forma bruta. Gostaríamos de agradecer aos autores de [0] por disponibilizarem os dados!


### Referências

[0] Le, Q., & Mikolov, T. (2014, January). Distributed representations of sentences and documents. In International conference on machine learning (pp. 1188-1196). Link: https://cs.stanford.edu/~quocle/paragraph_vector.pdf

[1] Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).

### Iniciando...

Em primeiro lugar, vamos carregar os pacotes que faremos uso:

In [2]:
!pip install ftfy
!pip install gensim

     |████████████████████████████████| 71kB 4.0MB/s 
  Created wheel for ftfy: filename=ftfy-5.8-cp36-none-any.whl size=45612 sha256=feebd5c5610c053540354ebd6803efb8fa7ceb478c137851d9c2367fdde2ea8e
  Stored in directory: /root/.cache/pip/wheels/ba/c0/ef/f28c4da5ac84a4e06ac256ca9182fc34fa57fefffdbc68425b
Successfully built ftfy


In [3]:
#Para o uso geral
import random
import numpy as np
import pandas as pd
import copy 
import time
from scipy.stats import uniform
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import io

#Para o processamento de textos
from ftfy import fix_text
import string
import re
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

#Para Machine Learning e NLP
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

Abrindo dados:

In [4]:
%%time
download = requests.get("https://raw.githubusercontent.com/felipemaiapolo/hands-on_ic_ml/master/data_train.csv").content
data = pd.read_csv(io.StringIO(download.decode('utf-8')), sep=',')

CPU times: user 774 ms, sys: 248 ms, total: 1.02 s
Wall time: 4.7 s


In [5]:
data.head()

,review,positive
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1
3,This is easily the most underrated film inn th...,1
4,This is not the typical Mel Brooks film. It wa...,1


Vamos checar se há textos duplicados e especular se deveriam estar duplicados ou não:

In [6]:
data.loc[data.duplicated(subset='review', keep=False)==True].sort_values(by='review').head(10)

,review,positive
24826,'Dead Letter Office' is a low-budget film abou...,0
18434,'Dead Letter Office' is a low-budget film abou...,0
8122,".......Playing Kaddiddlehopper, Col San Fernan...",1
11731,".......Playing Kaddiddlehopper, Col San Fernan...",1
21968,"<br /><br />Back in his youth, the old man had...",0
21969,"<br /><br />Back in his youth, the old man had...",0
8567,A have a female friend who is currently being ...,1
8545,A have a female friend who is currently being ...,1
5063,"A longtime fan of Bette Midler, I must say her...",1
12112,"A longtime fan of Bette Midler, I must say her...",1


Pelo fato de as avaliações duplicadas não serem muito simples, é provável que as duplicações de devam a erros na coleta dos dados e não duplicações que poderiam de fato ocorrer. Então, vamos excluir uma das avaliações duplicadas:

In [7]:
data=data.drop_duplicates(subset='review', keep='first')
data.shape

(24888, 2)

Temos duas colunas, sendo que a primeira contém avaliações (escritas) sobre filmes e a segunda nos diz se aquela avaliação é positiva ou negativa - se 'positive'==1 para uma certa avaliação, então aquela avaliação tem sentimento positivo. Por outro lado, se 'positive'==0 para uma certa avaliação, então aquela avaliação tem sentimento negativo. Tendo duas classes, dizemos que temos um problema de classificação binária.

Vamos ver a distribuição de 'positive':

In [8]:
data.loc[:,'positive'].value_counts()

1    12461
0    12427
Name: positive, dtype: int64

Agora vamos criar as listas $X$ e $y$, que contém os textos e os marcadores, respectivamente:

In [9]:
X = data.loc[:,'review'].tolist()
y = np.array(data.loc[:,'positive'].tolist())

Vamos ver um dos textos:

In [10]:
X[5]

"This isn't the comedic Robin Williams, nor is it the quirky/insane Robin Williams of recent thriller fame. This is a hybrid of the classic drama without over-dramatization, mixed with Robin's new love of the thriller. But this isn't a thriller, per se. This is more a mystery/suspense vehicle through which Williams attempts to locate a sick boy and his keeper.<br /><br />Also starring Sandra Oh and Rory Culkin, this Suspense Drama plays pretty much like a news report, until William's character gets close to achieving his goal.<br /><br />I must say that I was highly entertained, though this movie fails to teach, guide, inspect, or amuse. It felt more like I was watching a guy (Williams), as he was actually performing the actions, from a third person perspective. In other words, it felt real, and I was able to subscribe to the premise of the story.<br /><br />All in all, it's worth a watch, though it's definitely not Friday/Saturday night fare.<br /><br />It rates a 7.7/10 from...<br />

Assim como feito no notebook "Introdução ao Doc2Vec", vamos definir uma função para a limpeza e padronização dos textos.

**MUITO IMPORTANTE:** é *muito* recomendado que a função de limpeza utilizada no uso do modelo de representação, i.e. Doc2Vec, seja **idêntica** à aquele utilizada no momento do treinamento do modelo de representação.

In [11]:
def clean(text):
    txt=text.replace("<br />"," ") #retirando tags
    txt=fix_text(txt) #consertando Mojibakes (Ver https://pypi.org/project/ftfy/)
    txt=txt.lower() #passando tudo para minúsculo
    txt=txt.translate(str.maketrans('', '', string.punctuation)) #retirando toda pontuação
    txt=txt.replace(" — ", " ") #retirando hífens
    txt=re.sub("\d+", ' <number> ', txt) #colocando um token especial para os números
    txt=re.sub(' +', ' ', txt) #deletando espaços extras
    return txt

Limpando e padronizando os textos:

In [12]:
%%time
X = [clean(x) for x in X]
X[5]

CPU times: user 25.8 s, sys: 23.3 ms, total: 25.8 s
Wall time: 25.8 s


In [13]:
print(X[5])

this isnt the comedic robin williams nor is it the quirkyinsane robin williams of recent thriller fame this is a hybrid of the classic drama without overdramatization mixed with robins new love of the thriller but this isnt a thriller per se this is more a mysterysuspense vehicle through which williams attempts to locate a sick boy and his keeper also starring sandra oh and rory culkin this suspense drama plays pretty much like a news report until williams character gets close to achieving his goal i must say that i was highly entertained though this movie fails to teach guide inspect or amuse it felt more like i was watching a guy williams as he was actually performing the actions from a third person perspective in other words it felt real and i was able to subscribe to the premise of the story all in all its worth a watch though its definitely not fridaysaturday night fare it rates a <number> from the fiend 


Tokenizando os textos:

In [14]:
X = [x.split() for x in X]

In [15]:
X[5]

['this',
 'isnt',
 'the',
 'comedic',
 'robin',
 'williams',
 'nor',
 'is',
 'it',
 'the',
 'quirkyinsane',
 'robin',
 'williams',
 'of',
 'recent',
 'thriller',
 'fame',
 'this',
 'is',
 'a',
 'hybrid',
 'of',
 'the',
 'classic',
 'drama',
 'without',
 'overdramatization',
 'mixed',
 'with',
 'robins',
 'new',
 'love',
 'of',
 'the',
 'thriller',
 'but',
 'this',
 'isnt',
 'a',
 'thriller',
 'per',
 'se',
 'this',
 'is',
 'more',
 'a',
 'mysterysuspense',
 'vehicle',
 'through',
 'which',
 'williams',
 'attempts',
 'to',
 'locate',
 'a',
 'sick',
 'boy',
 'and',
 'his',
 'keeper',
 'also',
 'starring',
 'sandra',
 'oh',
 'and',
 'rory',
 'culkin',
 'this',
 'suspense',
 'drama',
 'plays',
 'pretty',
 'much',
 'like',
 'a',
 'news',
 'report',
 'until',
 'williams',
 'character',
 'gets',
 'close',
 'to',
 'achieving',
 'his',
 'goal',
 'i',
 'must',
 'say',
 'that',
 'i',
 'was',
 'highly',
 'entertained',
 'though',
 'this',
 'movie',
 'fails',
 'to',
 'teach',
 'guide',
 'inspect',


### Utilizando Doc2Vec e modelos supervisionados para Análise de Sentimentos

Abrindo Doc2Vec pré-treinado (treinado no notebook "Introdução ao Doc2Vec"):

In [17]:
%%time
d2v = Doc2Vec.load('doc2vec')  

CPU times: user 725 ms, sys: 64 ms, total: 789 ms
Wall time: 790 ms


Vamos definir uma função para obtermos as representações vetoriais dos textos. 

No momento do treinamento do Doc2Vec, geramos representações somente para os textos que foram utilizados para aquele fim. Então teremos que inferir as representações para os novos textos, que utilizaremos agora. A inferência é feita utilizando a descida pela gradiente, congelando a rede neural do Doc2Vec e atualizando somente os pesos referentes ao novos textos. Na função abaixo, fixamos uma semente (*seed*) afim de garantir resultados reprodutíveis e definimos que a descida pelo gradiente dê 20 passos:

In [18]:
def emb(txt, model, normalize=False): 
    model.random.seed(42)
    x=model.infer_vector(txt, steps=20)
    
    if normalize: return(x/np.sqrt(x@x))
    else: return(x)

Obtendo as representações vetoriais para os textos:

In [19]:
%%time
X = [emb(x, d2v) for x in X] 
X = np.array(X)

CPU times: user 4min 14s, sys: 275 ms, total: 4min 14s
Wall time: 4min 14s


In [20]:
X[5]

array([-4.9251288e-01, -7.4880034e-01, -4.6935514e-01, -3.5521227e-01,
       -9.6032226e-01, -2.2638313e-01, -1.7565851e-01,  2.0557454e-01,
       -3.8599452e-01, -1.0547549e+00, -5.2651966e-01,  2.6664075e-01,
       -3.1290192e-02,  8.2340263e-02,  5.2198941e-01,  4.1168782e-01,
        6.0595465e-01, -2.7571696e-01, -6.1859661e-01, -1.0638729e+00,
        2.3582111e-01,  7.9060525e-01, -3.8301581e-01, -5.8371067e-01,
        1.1286757e-01,  3.8241524e-02, -7.4234474e-01,  4.7949597e-02,
       -1.2712264e-01,  6.4678907e-01,  4.0058026e-01, -2.8603878e-02,
       -8.9273268e-01,  1.1894169e+00,  3.7112080e-03,  4.5376581e-01,
       -5.7880771e-01,  8.3966472e-04,  3.3799714e-01, -9.8302364e-02,
        1.4078172e-01, -4.8945490e-01,  3.9696792e-01,  5.5010712e-01,
       -9.9452265e-02, -7.1044856e-01,  9.0012169e-01, -2.4865916e-01,
        7.7567583e-01, -3.2373193e-01], dtype=float32)

Agora que já temos $X$ e $y$ nos formatos usuais e que vocês já conhecem bem, podemos seguir para a etapa de classificação. Vamos ver os formatos dos dois arrays:

In [21]:
X.shape, y.shape

((24888, 50), (24888,))

Agora vamos treinar um modelo de Regressão Logística.

Destinando parte da base para teste:

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

Utilizando Random Search para escolher os melhores hiperparâmetros para o modelo de classificação Regressão Logística para a análise de sentimentos com base na métrica *roc_auc*:

In [24]:
%%time
logreg = LogisticRegression(solver='liblinear',random_state=42)
hyperparams = dict(C=np.linspace(0,10,100), 
                     penalty=['l2', 'l1'])
clf = RandomizedSearchCV(logreg, hyperparams, scoring='roc_auc', n_iter=50, cv=2, n_jobs=-1, random_state=0, verbose=2)
search_logreg = clf.fit(X_train, y_train)

search_logreg.best_params_, search_logreg.best_score_ 

Fitting 2 folds for each of 50 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:   17.5s finished


CPU times: user 619 ms, sys: 90 ms, total: 709 ms
Wall time: 17.8 s


In [25]:
search_logreg.best_params_, search_logreg.best_score_ 

({'C': 0.20202020202020202, 'penalty': 'l2'}, 0.8853808836106671)

Treinando modelos finais e vendo suas performances no conjunto de teste

In [26]:
logreg = LogisticRegression(C=search_logreg.best_params_['C'], 
                            penalty=search_logreg.best_params_['penalty'],
                            solver='liblinear', random_state=42)

In [27]:
logreg.fit(X_train, y_train)

print('AUC --- Log. Reg.: {:.4f}'.format(roc_auc_score(y_test, logreg.predict_proba(X_test)[:,1])))

AUC --- Log. Reg.: 0.8807
